In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/EMNLP_folder_4/dataset

/content/drive/.shortcut-targets-by-id/1ANtRsCBtHNhrdxlObGcNtnE59DOMZyhS/EMNLP_folder_4/dataset


In [ ]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

     |████████████████████████████████| 2.5MB 30.9MB/s 
     |████████████████████████████████| 3.3MB 51.2MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 1.2MB 37.9MB/s 


In [ ]:
from pandarallel import pandarallel

# Initialization
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Importing the Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
import spacy 
from spacy import displacy
#import neptune.new as neptune

from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset)

In [ ]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)
device = 'cuda'

In [ ]:
path_dataset = '/content/drive/MyDrive/EMNLP_folder_4/dataset/'
path_predictions_folder = '/content/drive/MyDrive/EMNLP_folder_4/Track_2_submission_files/'
save_model_folder = '/content/drive/MyDrive/EMNLP_folder_4/combined_dataset_model_files/'

In [ ]:
max_len_arg = 55
max_len_kp = 32
max_len_topic = 12
max_len_input = 128
model_with_no_token_types =['roberta', 'bart' ,'longformer','albert','electra','reformer','distilbert','scibert','deberta', 'xlnet', 'xlmroberta', 'bartlarge', 'robertalarge', 'xlmrobertalarge', 'bartlargecnn','debertalarge','xlnetlarge']

## Making the Combined Dataset for Predicting their Scores

In [ ]:
df_arg = pd.read_csv('/content/drive/MyDrive/EMNLP_Argmining2021/dataset/arguments_test.csv')
df_kp = pd.read_csv('/content/drive/MyDrive/EMNLP_folder_4/gen_key_points_predictions/track_2_key_points.csv')

print(df_arg.shape,df_kp.shape)
df_kp.head()

(723, 4) (26, 4)


,key_point_id,key_point,topic,stance
0,kp_0_9,Child vaccinations are necessary to protect ag...,Routine child vaccinations should be mandatory,1
1,kp_0_10,A mandatory vaccination is important to protec...,Routine child vaccinations should be mandatory,1
2,kp_0_11,Children should be vaccinated against diseases...,Routine child vaccinations should be mandatory,1
3,kp_0_12,Children should be vaccinated against diseases,Routine child vaccinations should be mandatory,1
4,kp_0_13,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,-1


In [ ]:
df_arg.head()

,arg_id,argument,topic,stance
0,arg_0_0,Routine child vaccinations isn't mandatory sin...,Routine child vaccinations should be mandatory,-1
1,arg_0_1,Routine child vaccinations should not be manda...,Routine child vaccinations should be mandatory,-1
2,arg_0_2,Routine child vaccinations should not be neces...,Routine child vaccinations should be mandatory,-1
3,arg_0_3,A vaccine that has not been sufficiently teste...,Routine child vaccinations should be mandatory,-1
4,arg_0_4,As long as vaccines are not free of side effec...,Routine child vaccinations should be mandatory,-1


In [ ]:
def give_tuples(df_arguments,df_key_points):

  arguments = []
  key_points = []
  topics = []
  stance = []
  arg_ids =[]
  kp_ids = []

  for i,j,k,l in tqdm(zip(df_arguments['argument'],df_arguments['topic'],df_arguments['stance'],df_arguments['arg_id'])):
    df = df_key_points[(df_key_points['topic'] == j) & (df_key_points['stance'] == k)]
    #print()
    #print(j)
    #print(k)

    if(df.shape[0]>0):
      for o,p in zip(df['key_point'],df['key_point_id']):
        arguments.append(i)
        key_points.append(o)
        topics.append(j)
        stance.append(k)
        arg_ids.append(l)
        kp_ids.append(p)

    

  return arguments,key_points,topics, stance,arg_ids,kp_ids

In [ ]:
args,kps,topics,stance,arg_ids,kp_ids = give_tuples(df_arg,df_kp)

In [ ]:
df_test = pd.DataFrame({'arg_id':arg_ids,'key_point_id':kp_ids,'argument':args, 'keypoint':kps, 'topic' : topics , 'stance': stance})

df_test.head()

,arg_id,key_point_id,argument,keypoint,topic,stance
0,arg_0_0,kp_0_13,Routine child vaccinations isn't mandatory sin...,Parents are not adequately informed about the ...,Routine child vaccinations should be mandatory,-1
1,arg_0_0,kp_0_14,Routine child vaccinations isn't mandatory sin...,Not everyone has the ability to go to the doct...,Routine child vaccinations should be mandatory,-1
2,arg_0_0,kp_0_15,Routine child vaccinations isn't mandatory sin...,Parents should be permitted to choose the vacc...,Routine child vaccinations should be mandatory,-1
3,arg_0_0,kp_0_16,Routine child vaccinations isn't mandatory sin...,Child vaccines can be given to the wrong person,Routine child vaccinations should be mandatory,-1
4,arg_0_0,kp_0_17,Routine child vaccinations isn't mandatory sin...,Not everyone has the ability to pay for vaccines,Routine child vaccinations should be mandatory,-1


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
test_list = []

for i,j,k in  zip(df_test['keypoint'],df_test['argument'],df_test['topic']):
  test_list.append(i+j+k)

In [ ]:
df_test['kp_arg_topic'] = test_list

In [ ]:
def give_noun_features(sentence):
  sent_dep_list =[]
  doc = nlp(sentence)
  for token in doc:
    sent_dep_list.append(str(token.head.pos_))

  return sent_dep_list

In [ ]:
df_test['noun_features'] = df_test['kp_arg_topic'].parallel_apply(lambda x : give_noun_features(x))

   0.00%                                          |        0 /     1565 |      
   0.00%                                          |        0 /     1565 |      M   0.96%                                          |       15 /     1565 |      
   0.00%                                          |        0 /     1565 |      M   1.73%                                          |       27 /     1565 |      
   0.96%                                          |       15 /     1565 |      M   2.68% :                                        |       42 /     1565 |      
   1.73%                                          |       27 /     1565 |      M   3.51% :                              

In [ ]:
train_noun_tokens_list = []

for i in tqdm(df_test['noun_features']):
  for j in i:
    if j not in train_noun_tokens_list:
      train_noun_tokens_list.append(j)

print(len(train_noun_tokens_list))


15


In [ ]:
train_noun_tokens_list = ['NOUN',
 'VERB',
 'ADP',
 'AUX',
 'ADJ',
 'INTJ',
 'PRON',
 'PROPN',
 'ADV',
 'DET',
 'SCONJ',
 'PART',
 'NUM',
 'SYM',
 'X',
 'PUNCT',
 'CCONJ']

In [ ]:
## Giving the same encoding that was given for train set
noun_dictionary= {}
noun_dictionary['VERB'] = 1
noun_dictionary['NOUN'] = 2
noun_dictionary['AUX'] = 3
noun_dictionary['ADP'] = 4
noun_dictionary['ADJ'] = 5
noun_dictionary['PROPN'] = 6
noun_dictionary['SCONJ'] = 7
noun_dictionary['ADV'] = 8
noun_dictionary['PRON'] = 9
noun_dictionary['DET'] = 10
noun_dictionary['NUM'] = 11
noun_dictionary['PART'] = 12
noun_dictionary['PUNCT'] = 13
noun_dictionary['SYM'] = 14
noun_dictionary['X'] = 15
noun_dictionary['INTJ'] = 16
noun_dictionary['CCONJ'] = 17

In [ ]:
def give_encoded_noun_list(sent):
  encoded_list = []

  for i in sent:
    encoded_list.append(noun_dictionary[i])

  return encoded_list

In [ ]:
df_test['encoded_noun_features'] = df_test['noun_features'].parallel_apply(lambda x : give_encoded_noun_list(x) )

   0.00%                                          |        0 /     1565 |      
   0.00%                                          |        0 /     1565 |      M   0.96%                                          |       15 /     1565 |      
   0.00%                                          |        0 /     1565 |      M   5.75% ::                                       |       90 /     1565 |      
   0.96%                                          |       15 /     1565 |      M 100.00% :::::::::::::::::::::::::::::::::::::::: |     1565 /     1565 |      
   5.05% ::                                       |       79 /     1565 |      M 100.00% :::::::::::::::::::::::::::::::

In [ ]:
MAX_NOUN_ENCODED_FEATURES_LEN = 66

In [ ]:
def give_padding(sent):

  var_len = len(sent)
  padding_len = 66-var_len
  padding = [0]*padding_len
  sent = sent + padding
  
  return sent

In [ ]:
df_test['encoded_noun_features'] = df_test['encoded_noun_features'].parallel_apply(lambda x : give_padding(x) )

   0.00%                                          |        0 /     1565 |      
   0.00%                                          |        0 /     1565 |      M   0.00%                                          |        0 /     1565 |      
   0.96%                                          |       15 /     1565 |      M   0.96%                                          |       15 /     1565 |      
   6.58% ::                                       |      103 /     1565 |      M   6.13% ::                                       |       96 /     1565 |      
 100.00% :::::::::::::::::::::::::::::::::::::::: |     1565 /     1565 |      M 100.00% :::::::::::::::::::::::::::::::

In [ ]:
def make_dataset(tokenizer, args,kps,topics,features, labels, max_len_input, model_with_no_token_types = model_with_no_token_types, model_name='roberta'):
    
    all_input_ids = []
    all_token_type_ids = []
    all_attention_masks = []
    all_labels = [] 
    all_features=[]
    
    for arg,kp,topic,feature,label in zip(args,kps,topics,features,labels) :

        arg = re.sub('[^a-zA-Z]', ' ', arg)
        kp = re.sub('[^a-zA-Z]', ' ', kp)
        topic = re.sub('[^a-zA-Z]', ' ', topic)

        url = re.compile(r'https?://\S+|www\.\S+')
        arg = url.sub(r'',arg)
        kp = url.sub(r'',kp)
        topic = url.sub(r'',topic)
        
        html=re.compile(r'<.*?>')
        arg = html.sub(r'',arg)
        kp = html.sub(r'',kp)
        topic = html.sub(r'',topic)


        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        
        arg = emoji_pattern.sub(r'',arg)
        kp = emoji_pattern.sub(r'',kp)
        topic = emoji_pattern.sub(r'',topic)

        if model_name in model_with_no_token_types:

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          #all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
          all_features.append(feature)

        else :

          encoded_input = tokenizer(kp+arg+topic,max_length = max_len_input, padding='max_length')
          all_input_ids.append(encoded_input['input_ids'])
          all_attention_masks.append(encoded_input['attention_mask'])
          all_token_type_ids.append(encoded_input['token_type_ids'])
          all_labels.append(label)
          all_features.append(feature)
          
    if model_name in model_with_no_token_types:
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_features = torch.tensor(all_features).squeeze()
      all_labels = torch.tensor(all_labels)
      
      dataset = TensorDataset(all_input_ids, all_attention_masks,all_features, all_labels)

    else :
      all_input_ids = torch.tensor(all_input_ids).squeeze()
      all_token_type_ids = torch.tensor(all_token_type_ids).squeeze()
      all_attention_masks = torch.tensor(all_attention_masks).squeeze()
      all_features = torch.tensor(all_features).squeeze()
      all_labels = torch.tensor(all_labels) 

      dataset = TensorDataset(all_input_ids,all_token_type_ids, all_attention_masks,all_features, all_labels)

    return dataset

In [ ]:
model_name = 'debertalarge'
model_path = 'microsoft/deberta-large'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
test_dataset = make_dataset(tokenizer, df_test['argument'], df_test['keypoint'], df_test['topic'],df_test['encoded_noun_features'], df_test['stance'], max_len_input, model_with_no_token_types, model_name=model_name)

# Model Architecture

In [ ]:
class NonPoolerTransformer(nn.Module):

    def __init__(self):
        super(NonPoolerTransformer, self).__init__()
        
        #Instantiating Pre trained model object 
        self.model_layer = AutoModel.from_pretrained(model_path)

        #for p in self.model_layer.parameters():
        #  p.requires_grad = False
        
        #Layers
        #834 or 1090
        self.dense_layer_1 = nn.Linear(1090, 256)
        self.dropout = nn.Dropout(0.4)
        self.dense_layer_2 = nn.Linear(256, 128)
        self.dropout_2 = nn.Dropout(0.4)
        self.cls_layer = nn.Linear(128, 1, bias = True)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input_ids, attention_masks,features):

        hidden_state = self.model_layer(input_ids=input_ids, attention_mask=attention_masks)[0]
        #print(pooled_output)
        pooled_output = hidden_state[:, 0]

        concat = torch.cat((pooled_output,features),dim =1)

        x = self.dense_layer_1(concat)
        x = self.dropout(x)
        x_1 = self.dense_layer_2(x)
        x_2 = self.dropout_2(x_1)

        
        logits = self.cls_layer(x_2)
        output = self.sigmoid(logits)

        return output

In [ ]:
model = NonPoolerTransformer()

Some weights of the model checkpoint at microsoft/deberta-large were not used when initializing DebertaModel: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = model.to(device)

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
EPOCHS = 3
ACCUMULATION_STEPS = 2
DROPOUT = 0.4

In [ ]:
def predict_model(test_dataset,df, save_model,model_name):
  preds = []

  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
  model=torch.load(save_model_folder + save_model +'.pt')

  with torch.no_grad():
    
    epoch_iterator = tqdm(test_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.eval()

      if model_name in model_with_no_token_types:
        b_input_ids, b_input_mask,b_features, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        ypred = model(b_input_ids, b_input_mask,b_features)
      else:
        b_input_ids,b_token_type, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device),batch[4].to(device)
        ypred = model(b_input_ids, b_input_mask,b_features)

      ypred = ypred.cpu().numpy()
      ypred = np.hstack(ypred)

      preds.append(ypred)

  args = df['arg_id']
  kps = df['key_point_id']
  all_preds = []

  for i in tqdm(range(len(preds))):
    for p in preds[i]:
      all_preds.append(p)

  pred_file = pd.DataFrame({"arg_id" : args, "key_point_id": kps, "score": all_preds})

  args = {}
  kps = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg] = {}

  for arg,kp,score in zip(pred_file['arg_id'],pred_file['key_point_id'],pred_file['score']):
    args[arg][kp] = score

  with open(path_predictions_folder + save_model + '_' + 'predictions.p.', 'w') as fp:
    fp.write(json.dumps(args))
    fp.close()

  print("The predictions are stored in the file : "+ path_predictions_folder  + save_model + '_' + 'predictions.p.')

In [ ]:
save_model  = 'debertalarge_noun_feature__epoc_3_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic_feature'

In [ ]:
predict_model (test_dataset,df_test, save_model,model_name = 'debertalarge')


The predictions are stored in the file : /content/drive/MyDrive/EMNLP_folder_4/Track_2_submission_files/debertalarge_noun_feature__epoc_3_lr_1e-05_b_s_16_accumulation_steps_2_input_type_kp_arg_topic_feature_predictions.p.
